In [1]:
import cv2
import os
import sys
from datetime import datetime
from ultralytics import YOLO
import matplotlib.pyplot as plt

cwd = os.getcwd()
delimiter = "\\" if "\\" in cwd else "/"
repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
sys.path.append(repoPath + "dataImport/")
import vidUtils
from vidUtils import getFrame, getCap
import rwWorkingTSDf
from rwWorkingTSDf import readWorkingTSDF, writeWorkingTSDf, getWorkingTSDfPath

In [2]:
model = YOLO("yolo11x-pose.pt")

In [3]:
deviceDescriptor = ["abhik", "bedroompi", "abhik", "piCam-raspberryPi5-Camv3120", "mp4", "piVidCap"]
outputDescriptor = ["abhik", "bedroompi", "ultralytics", "piCam-raspberryPi5-Camv3120", "17pose", "yolo11x-pose"]

In [4]:
vidDf = readWorkingTSDF(deviceDescriptor)
import json

newDfRows = []
for i in range(100):
    row = vidDf.iloc[i]
    ts = row.name
    videoIndex = row['videoIndex']
    videoStartTime = row['videoStartTime']
    videoEndTime = row['videoEndTime']
    frame = getFrame(deviceDescriptor, videoStartTime, videoEndTime, videoIndex)
    result = model(frame, verbose=False)
    toSave = json.dumps({'xyn': result[0].keypoints.xyn.tolist(), 'conf': result[0].keypoints.conf.tolist()})
    newDfRows.append([ts, toSave])


read in 30466 rows from 1 files, retruning 30466 rows


In [5]:
import pandas as pd
newDf = pd.DataFrame(data=newDfRows, columns=['sampleDT', 'pose'])
newDf = newDf.set_index('sampleDT')
newDf

,pose
sampleDT,
2025-03-13 23:03:00.000596+00:00,"{""xyn"": [[[0.548828661441803, 0.53396099805831..."
2025-03-13 23:03:00.100148+00:00,"{""xyn"": [[[0.5484716296195984, 0.5321729779243..."
2025-03-13 23:03:00.201352+00:00,"{""xyn"": [[[0.5490943193435669, 0.5313501954078..."
2025-03-13 23:03:00.303282+00:00,"{""xyn"": [[[0.5493457913398743, 0.5299937725067..."
2025-03-13 23:03:00.402564+00:00,"{""xyn"": [[[0.5500948429107666, 0.5293601155281..."
...,...
2025-03-13 23:03:09.500148+00:00,"{""xyn"": [[[0.5534456372261047, 0.5194787979125..."
2025-03-13 23:03:09.600136+00:00,"{""xyn"": [[[0.5525461435317993, 0.5217380523681..."
2025-03-13 23:03:09.700147+00:00,"{""xyn"": [[[0.5516005158424377, 0.5251188278198..."


In [6]:
writeWorkingTSDf(outputDescriptor, newDf)

hashes match for 2025-03-13T230300,000596+0000_2025-03-13T230309,900155+0000_821a0a3a_.parquet.gzip


In [7]:
# "SELECT * FROM 'data/video_*.parquet.gzip'"
#maybe I shuold use snappy?

In [16]:
import duckdb


wdp = getWorkingTSDfPath(outputDescriptor)
sTS = newDf.iloc[2].name.isoformat()
eTS = newDf.iloc[97].name.isoformat()

query = """
SELECT * FROM read_parquet('?*')
WHERE sampleDT BETWEEN ? AND ?
"""
result = duckdb.query(query, [wdp, sTS, eTS]).df()
print(result)

TypeError: query(): incompatible function arguments. The following argument types are supported:
    1. (query: object, *, alias: str = '', params: object = None, connection: duckdb.DuckDBPyConnection = None) -> duckdb.duckdb.DuckDBPyRelation

Invoked with: "\nSELECT * FROM read_parquet('?*')\nWHERE sampleDT BETWEEN ? AND ?\n", ['/home/chowder/Documents/workingData/abhik_bedroompi_ultralytics_piCam-raspberryPi5-Camv3120_17pose_yolo11x-pose/', '2025-03-13T23:03:00.201352+00:00', '2025-03-13T23:03:09.700147+00:00']

In [18]:
import duckdb
import pandas as pd

# Example: Extracting timestamps from a DataFrame index
sTS = newDf.iloc[3].name
eTS = newDf.iloc[97].name

# Open a DuckDB connection
con = duckdb.connect()

# Query execution using parameterized input
query = f"""
SELECT * FROM read_parquet('{wdp}*')
WHERE sampleDT BETWEEN ? AND ?
"""

result = con.execute(query, [sTS, eTS]).df().set_index('sampleDT')
print(sTS)
print(eTS)
print(result)

2025-03-13 23:03:00.303282+00:00
2025-03-13 23:03:09.700147+00:00
                                                                               pose
sampleDT                                                                           
2025-03-13 16:03:00.303282-07:00  {"xyn": [[[0.5493457913398743, 0.5299937725067...
2025-03-13 16:03:00.402564-07:00  {"xyn": [[[0.5500948429107666, 0.5293601155281...
2025-03-13 16:03:00.500204-07:00  {"xyn": [[[0.551284670829773, 0.52800923585891...
2025-03-13 16:03:00.600374-07:00  {"xyn": [[[0.5559868216514587, 0.5183365941047...
2025-03-13 16:03:00.700461-07:00  {"xyn": [[[0.562384307384491, 0.50334841012954...
...                                                                             ...
2025-03-13 16:03:09.300135-07:00  {"xyn": [[[0.5577629208564758, 0.5098487734794...
2025-03-13 16:03:09.400139-07:00  {"xyn": [[[0.5545237064361572, 0.5173035264015...
2025-03-13 16:03:09.500148-07:00  {"xyn": [[[0.5534456372261047, 0.5194787979125...
2025-03-13

In [20]:
import duckdb
import pandas as pd

# Example: Extracting timestamps from a DataFrame index
sTS = newDf.iloc[3].name.isoformat()
eTS = newDf.iloc[97].name.isoformat()

# Open a DuckDB connection
con = duckdb.connect()

# Query execution using parameterized input
query = f"""
SELECT * FROM read_parquet('{wdp}*')
WHERE sampleDT BETWEEN '{sTS}' AND '{eTS}'
"""

result = con.execute(query).df().set_index('sampleDT')
print(sTS)
print(eTS)
print(result)

2025-03-13T23:03:00.303282+00:00
2025-03-13T23:03:09.700147+00:00
                                                                               pose
sampleDT                                                                           
2025-03-13 16:03:00.303282-07:00  {"xyn": [[[0.5493457913398743, 0.5299937725067...
2025-03-13 16:03:00.402564-07:00  {"xyn": [[[0.5500948429107666, 0.5293601155281...
2025-03-13 16:03:00.500204-07:00  {"xyn": [[[0.551284670829773, 0.52800923585891...
2025-03-13 16:03:00.600374-07:00  {"xyn": [[[0.5559868216514587, 0.5183365941047...
2025-03-13 16:03:00.700461-07:00  {"xyn": [[[0.562384307384491, 0.50334841012954...
...                                                                             ...
2025-03-13 16:03:09.300135-07:00  {"xyn": [[[0.5577629208564758, 0.5098487734794...
2025-03-13 16:03:09.400139-07:00  {"xyn": [[[0.5545237064361572, 0.5173035264015...
2025-03-13 16:03:09.500148-07:00  {"xyn": [[[0.5534456372261047, 0.5194787979125...
2025-03-13